<a href="https://colab.research.google.com/github/Antony-gitau/machine_learning_playground/blob/main/Neurons_with_recurrence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I am following the [MIT 6.S191 lecture on recurrent neural network](https://youtu.be/ySEx_Bqxvvo) and taking some notes and here I document them.

Sequence modelling applications:
- machine translation 
- image captioning
- semantic classification


Neuron with recurrence
- RNN

pseudocode of an RNN

1. Define the rnn;
my_rnn = RNN()
2. iterate through all the inputs
3. calculate and update the hidden state using an activation function
4. generate a predicted output.

design criteria for developing networks for sequence modelling:
- handle variable lengths
- track long dependencies
- maintain information about the order of the sequence
- share parameters across the sequence

example: 
predicting the next word.

1. represent language to a neural network
- represent words as numerical representation. 

one way to represent words as input vectors of a neural network, we use a one hot encoding technique. By one hot encoding we mean, taking a count of every word in a single vector and identifying the word with a 1 and 0 everywhere else. e.g [0,1,0,0] is a one hot vector of a word in the second index (that is appearing second on the count of words in the sequence)

2. Training and learning through neural networks

- backpropagation through time.

challenges:
1. exploding gradients
- the gradient gets bigger and bigger until its unfeaseble to calculate it, and by extension, training a model becomes unstable.
2. vanishing gradients
- the gradient on the other hand gets smaller and smaller, until it becomes insignificant.

tricks to overcome the challenges:
1. changing activation functions
e.g ReLU is an a function that prevents the gradient from shrinking
2. parameter initialization
3. introducing gated cells.
select flow of information in the neural network. like the LSTMs

applications and limitations of RNN

Music generation
- Design an RNN that can predict the next musical note.

limitation
- encoding bottleneck
- no easy parallelization techniques
- not that long memory for quite long sequences, like the 10,000s of words

Attention is all you need:
- attend to the most import part of an input example.
- extract the features deserve the highest attention.


